# Obtain Forecasts Using Exponential Smoothing

***

In [14]:
# load modules
import statsmodels.tsa as sm
import numpy as np

In [15]:
# import weekly finance series, skipping column names
ts = np.genfromtxt("../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)

***

In [17]:
# exponential smoothing function
# can be used for SES + trend/damped trend + seasonal
ES = sm.ExponentialSmoothing(ts[0,:100])

AttributeError: module 'statsmodels.tsa' has no attribute 'ExponentialSmoothing'